In [8]:
from random import sample
from fpylll import *
#parameter

N = 13;p=3;q =127 ;d=5 ;# q > p(6d+1) prime # p>2 prime gcd(N,q)=gcd(p,q)=1
R.<x> = ZZ['x']

In [9]:
def random_poly(o,mo):
 
    s=[1]*o+[-1]*mo+[0]*(N-o-mo)
    shuffle(s)
    return R(s)

def Convolution_in_R(ff,gg,NN):
    return (ff*gg)%(x^NN-1)

def Convolution_in_R_p(ff,gg,NN,pp):
    hh =ff*gg
    T = R.change_ring(Integers(pp)).quotient(x^NN-1)
    return  R(lift(T(hh)))  

def Invertmodprime(ff,pp,NN):    
    T = R.change_ring(Integers(pp)).quotient(x^NN-1)
    return R(lift(1/T(ff)))

def center_lift(ff,pp):
    flist = ff.list()
    ph = pp//2
    for i in range(len(flist)):
        tmp = int(flist[i])%pp
        if tmp > ph:
            tmp -= pp
        flist[i] = tmp
    return R(flist)

def keygen():
    f = random_poly(d+1,d)
    g = random_poly(d,d)
    try :
        fq = Invertmodprime(f,q,N)
    except:
        raise Exception
    h = Convolution_in_R_p(fq,g,N,q)
    return h,f,g

def enc(m):
    phi = random_poly(d,d)
    e_q = Convolution_in_R_p(h,p*phi,N,q)
    e = e_q + m
    return e%q
    
def dec(e_q,ff):
    a = Convolution_in_R_p(ff,e_q,N,q)
    a_c = center_lift(a,q)
    fp = Invertmodprime(ff,p,N)
    b = Convolution_in_R_p(a_c,fp,N,p)
    b = center_lift(b,p)
    return b


In [10]:
h,f,g = keygen()
m = R([randint(-(p-1)/2,(p-1)/2) for i in range(N)])
e = enc(m)
dec_m = dec(e,f)
if dec_m == m:
    print("OK")
    print("m is :",m)
    print("e is :",e)
    print("f is :",f)
    print("d is :",dec_m)
else:
    print(m)
    print(e)
    print("Error")


OK
m is : x^12 + x^10 - x^8 + x^7 + x^5 + x^3 + x + 1
e is : 57*x^12 + 33*x^11 + 87*x^10 + 6*x^9 + 80*x^8 + 19*x^7 + 121*x^6 + 33*x^5 + 91*x^4 + 3*x^3 + 3*x^2 + 89*x + 19
f is : -x^12 + x^11 - x^10 + x^8 - x^7 + x^6 - x^5 + x^4 - x^3 + x + 1
d is : x^12 + x^10 - x^8 + x^7 + x^5 + x^3 + x + 1


In [15]:
#Lattice attack

def NTRU_matrix(hh,qq,NN):
    H = Matrix(ZZ,NN,NN)
    for i in range(NN):
        for j in range(NN):
            H[i,j] = hh[j]
        hh*=x
        hh = hh%(x^NN -1)
    
    NTRU_B = Matrix(ZZ,2*NN,2*NN)
    for i in range(NN):
        NTRU_B[i,i] = 1
        NTRU_B[i+NN,i+NN] = qq
        for j in range(NN):
            NTRU_B[i,j+NN] = H[i,j]
    return NTRU_B

def L_LLL(B):
    B = B. LLL ()
    return B
 



def NTRU_lattice_attack(LL,dd,hh,NN):
    for i in range(NN):
        G = [0]*N;sum_b=0;flag = True;
        for j in range(NN):
            G[j] = LL[i,j+NN]
            if abs(G[j]) <= 1: #ノルム
                sum_b += abs(G[j])
            else:
                flag = False
        if flag and sum_b == 2*dd:
            F = [0]*NN
            for k in range(NN):
                F[k] = LL[i,k]
            f_atk = R(F);g_atk=R(G);
            
            if center_lift(Convolution_in_R_p(f_atk,hh,NN,q),q) == g_atk:
                print("f_atk:"+str(f_atk))
                return True,f_atk,g_atk
            
    return False,0,0
            

In [16]:
BB = NTRU_matrix(h,q,N)
L = L_LLL(BB)
flag,f_atk,g_atk = NTRU_lattice_attack(L,d,h,N)
if flag:
    atk_m = dec(e,f_atk)
    if m == atk_m:
        print("TRUE!!")
        print("dec_m :"+str(atk_m))
else:
    print("FALSE!!")

f_atk:-x^12 + x^11 - x^10 + x^9 - x^7 + x^6 - x^5 + x^4 - x^3 + x^2 - 1
TRUE!!
dec_m :x^12 + x^10 - x^8 + x^7 + x^5 + x^3 + x + 1
